In [1]:
from google.colab import drive
drive.mount('/content/gdrive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3aietf%3awg%3aoauth%3a2.0%3aoob&response_type=code&scope=email%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdocs.test%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive%20https%3a%2f%2fwww.googleapis.com%2fauth%2fdrive.photos.readonly%20https%3a%2f%2fwww.googleapis.com%2fauth%2fpeopleapi.readonly

Enter your authorization code:
··········
Mounted at /content/gdrive


In [0]:
import os

In [0]:
os.chdir('gdrive/My Drive/data')

In [6]:
!pip install ktrain

     |████████████████████████████████| 25.2MB 130kB/s 
     |████████████████████████████████| 421.8MB 41kB/s 
     |████████████████████████████████| 6.7MB 49.5MB/s 
     |████████████████████████████████| 983kB 48.5MB/s 
     |████████████████████████████████| 245kB 53.3MB/s 
     |████████████████████████████████| 645kB 51.2MB/s 
     |████████████████████████████████| 471kB 49.9MB/s 
     |████████████████████████████████| 450kB 51.8MB/s 
     |████████████████████████████████| 3.9MB 57.1MB/s 
     |████████████████████████████████| 1.1MB 53.1MB/s 
     |████████████████████████████████| 890kB 51.1MB/s 
     |████████████████████████████████| 3.8MB 46.9MB/s 
  Created wheel for ktrain: filename=ktrain-0.15.2-cp36-none-any.whl size=25242693 sha256=7fa14b7079370c551c433efad00c542e8f66edde8325170a79c4e5089fbc85a7
  Stored in directory: /root/.cache/pip/wheels/d5/d3/86/9ac5218077f10c3809c42c8a2b4b5aef2d29fbdb6838812ee9
  Created wheel for keras-bert: filename=keras_bert-0.81.0-cp36-no

In [0]:
import ktrain
from ktrain import text
import pandas as pd
import tensorflow as tf
import timeit
import argparse

from utils import load_data, train_dev_test, make_map_from_nested, word_vectors, trigram_vectors, all_vectors

In [0]:
def available_device(device_name):
    if device_name != '/device:GPU:0':
        print('Calculations will be made on CPU.')
    else:
        print('Calculations will be made on GPU.')
    return device_name

In [0]:
seed = 42
database, _ = load_data('./')

In [0]:
def available_device(device_name):
    if device_name != '/device:GPU:0':
        print('Calculations will be made on CPU.')
    else:
        print('Calculations will be made on GPU.')
    return device_name

In [0]:
database = database.drop('username', axis=1).rename(columns={"text": "TEXT", "class": "LABEL"})

In [0]:
def train(device_name, lr, epochs, batch, weight_save_path, df, text_column_df_name, possible_targets_list,
                predictor_save_path, maxlen=450):
    with tf.device(device_name):
        (x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(df, text_column_df_name,
                                                                           possible_targets_list,
                                                                           preprocess_mode='bert', maxlen=maxlen)
        model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)
        learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=batch)
        # learner.fit_onecycle(lr, epochs, checkpoint_folder=weight_save_path)
        learner.autofit(lr, epochs, checkpoint_folder=weight_save_path)
    predictor = ktrain.get_predictor(learner.model, preproc)
    predictor.save(predictor_save_path)

In [45]:
lr = 2e-5
batch = 12
epochs = 1
text_column_name = 'TEXT'
possible_targets_list = list(set((database.LABEL)))
weight_save_path = './bert_weights'
predictor_save_path = './my_predictor'
device_name = tf.test.gpu_device_name()
possible_device = available_device(device_name)

Calculations will be made on GPU.


In [54]:
(x_train, y_train), (x_test, y_test), preproc = text.texts_from_df(database, text_column_name,
                                                                   label_columns=['LABEL'], lang='ru',
                                                                   preprocess_mode='bert', maxlen=450)

downloading pretrained BERT model (multi_cased_L-12_H-768_A-12.zip)...
[██████████████████████████████████████████████████]
extracting pretrained BERT model...
done.

cleanup downloaded zip...
done.

preprocessing train...
language: ru


preprocessing test...
language: ru


In [55]:
model = text.text_classifier('bert', (x_train, y_train), preproc=preproc)

Is Multi-Label? False
maxlen is 450
done.


In [0]:
learner = ktrain.get_learner(model, train_data=(x_train, y_train), val_data=(x_test, y_test), batch_size=batch)

In [57]:
learner.autofit(lr, epochs, checkpoint_folder=weight_save_path)



begin training using triangular learning rate policy with max lr of 2e-05...
Train on 95089 samples, validate on 10566 samples
95089/95089 [==============================] - 12820s 135ms/sample - loss: 0.3472 - accuracy: 0.8958 - val_loss: 0.1650 - val_accuracy: 0.9498


In [0]:
predictor = ktrain.get_predictor(learner.model, preproc)
predictor.save(predictor_save_path)